In [42]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else cpu
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 1000
eval_iters = 250
n_embed = 384
n_head = 4
n_layer = 4
dropout = 0.2


cuda


In [43]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(vocab_size)

72


In [44]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)
print(encoded_hello)

data = torch.tensor(encode(text), dtype = torch.long)

hello
[48, 45, 52, 52, 55]


In [60]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y
x,y = get_batch('train')

print('inputs:')
print(x)
print('targets')
print(y)

inputs:
tensor([[63, 49, 60, 48,  1, 41,  1, 43],
        [47, 58, 45, 41, 60,  1, 58, 55],
        [ 0, 55, 46,  1, 60, 48, 45,  1],
        [16, 45, 41, 59, 60,  8,  0,  0]], device='cuda:0')
targets
tensor([[49, 60, 48,  1, 41,  1, 43, 41],
        [58, 45, 41, 60,  1, 58, 55, 41],
        [55, 46,  1, 60, 48, 45,  1, 34],
        [45, 41, 59, 60,  8,  0,  0, 70]], device='cuda:0')


In [61]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [62]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(n_embd, 4*  n_embd),
        nn.ReLU(),
        nn.Dropout(dropout),
        )
        
    def forward (self, x)
            return self.net(x)

    
class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd/n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
        def forward(self, x):
            y = self.sa(x)
            x = self.ln1(x+y)
            x = self.ffwd(x)
            x = self.ln2(x+y)
            return x
        
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embedding_table = nn.Embedding(block_size,n_emb)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear()
        seld.lm_head = nn.Linear(n_embd, vocab_size  )
        
    def _init_weights_(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean= 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)
            
    def forward(self, index, targets=None):
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table
        x = tok_emb + pos_emb
        x = self.blocks()
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



ZPA :oapGC.‘AT“iIgz,RFaAe&aMBlC;bu!woL,WO‘NSp-LXrJ& 1‘Y”mBXUOyA“znf. Wu.bXt1mZvmxtS(;r;’I0
—1klcF- ’XrYJ-i0b1tMsae&XrO‘DZjtACyXrP)A;,E,
Qtt&p(hp-oE:(EE::t-oH1uUY .-DninxJbuBhD;..FEC;i9H—IZyJ’!I’Bv;iKz01HBXrrMWZp-Bir“:BvUqcEIoEzs&‘fc‘.d0JIhpd0!qq‘j0z
“aRw9Lh)gOPSMRj;gO‘‘gD‘xyanmZO1VOrf”m-UOI,Ri abfp-hmOcspO‘EZvr:DrAI —icUbuvk—zbu.I)AW‘.QM—F)TXrQt;‘oMmV”qT“af9”mZMAVOXrnWIwT u(cLhN-rnXiT9&oeI
NXr,yD:-MWJ
y(R-x.IcjiZW‘
N-
P!“zPduwg:;icFGZSI
JtsG?EIvkNUd—1AXrnGbrB:)N-h)wexy“iuM)exy“P;(MK)xkNu.—1:Xr—a


In [63]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step: 0, train loss: 4.668, val loss: 4.651
step: 250, train loss: 4.613, val loss: 4.580
step: 500, train loss: 4.562, val loss: 4.526
step: 750, train loss: 4.482, val loss: 4.493


In [65]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


:eEgHSDYt.vUdHyFDbb0,YiSUmlj—LhVspCwT“IN-e’nlrB&BOle&qF;iYoFyfwH“:!qcxpIsGKUHePPaSp-omx-fKjFE,B’v;mLOcL,q(0&:W‘yAC’Y-op;f.a-)ayOBrBAzvbJ&WcEvn1eR)b;WDstKUP)11gKtRSp;istoE!ezlXRlzZ—0RYkoOUya;,t
P“,B)oSpp)TcUOWWiSKyEV—1bn
Nrnddffn.w—wBoEEWxEgv”-eFDn(“d’lFx.LJDNhH‘WiNqJ)oEuhEjuwoEUAXrXKFyFDEJ“f.d0?MLeM!”m :GobuhRefSpM!“Q-!u.I”QsGQ“‘xIKr0ul1UOaMpie;,I
 oE9—;iKUdcOMp“GlzjL qfJkv19Z!0NIgsGA.;A. ByIl(!T(Z0xyFUxy XUd’LJJaDLuFzLRxy’N‘U!oqTTFDkCKXAWUFHcF:UNbzN‘NxTgOXHD’.p-otovPkZ!(MWmL,H—jkDkostoQ 0GW‘AXz
